In [1]:
import scanpy as sc
import glob

import numpy as np
import pandas as pd
import pickle as pkl

from geosketch import gs
from fbpca import pca

import numpy as np
import pickle

import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
from uce_utils import read_uce_embed
#from cell_type_map_gpt import cell_type_mapping

import sys
sys.path.append('/dfs/user/yhr/cross-species-coarse/')
import matplotlib.pyplot as plt
from cs_utils import plotly_scatter

In [ ]:
all_datasets = np.load('all_datasets.npy')

In [3]:
all_paths = ["/lfs/local/0/yanay/cxg_npzs/" + f"{x}_counts.npz" for x in np.unique(all_datasets)]

In [4]:
cxg_shared_genes = np.load('../cxg_shared_genes.npy')

In [ ]:
## Non-h5ad route

#all_paths = glob.glob('/lfs/local/0/yanay/uce_all_proc_4layer/*')

## Only relevant datasets
all_paths = ["/lfs/local/0/yanay/cxg_npzs/" + f"{x}_counts.npz" for x in set(all_datasets)]

all_gene_names = []
success_loads = []
cxg_shared_genes = np.load('cxg_shared_genes.npy')

# Get the total number of rows in the final matrix
total_rows = sum(int(path.split('cells_')[-1].split('_')[0]) for path in all_paths)

# Create a memory-mapped array to store the final matrix
giant_matrix = np.memmap('/lfs/local/0/yhr/uce_all_proc/big_matrix_4layer_', shape=(total_rows, 1280), dtype='float32', mode='w+')

# Write data from each file to the memory-mapped array 
current_row = 0
for path in tqdm(all_paths, desc='Progress', unit='file'):
    try:
        ncells = int(path.split('cells_')[-1].split('_')[0])
        arr = np.memmap(path, shape=(ncells, 1280), dtype="float32", mode='r')
        giant_matrix[current_row:current_row + ncells, :] = arr
        current_row += ncells
    except:
        print('ERROR', path)

## Flush the memory-mapped array to ensure data is written to disk
giant_matrix.flush()


"""
# Load Cell Types
ct_path = f"/lfs/local/0/yanay/all_cell_types/{name}.pkl"
with open(ct_path, "rb") as f:
    cell_types = pkl.load(f)
"""

In [6]:
import scipy.sparse as sp

In [ ]:
#all_paths = glob.glob('/lfs/local/0/yanay/uce_all_proc_4layer/*')

## Only relevant datasets
all_paths = ['/lfs/local/0/yanay/cxg_h5s/'+x+'_proc.h5ad' for x in np.unique(all_datasets)]

all_h5ads = []
all_gene_names = []
success_loads = []
all_cell_types = []
cxg_shared_genes = np.load('cxg_shared_genes.npy')

failed_loads = []
for path in tqdm(all_paths, desc='Progress', unit='file'):
    
    #dataset_name = path.split('_')[-1].split('_counts')[0].split('.npz')[0]
    #raw_dataset_name = '/lfs/local/0/yanay/cxg_h5s/' + dataset_name +'_proc.h5ad'
    raw_dataset_name = path
    
    try:
        #ncells = int(path.split('cells_')[-1].split('_')[0])
        read_adata = sc.read_h5ad(raw_dataset_name)
        read_adata = read_adata[:, cxg_shared_genes]
        
        if read_adata.shape[1]<100:
            failed_loads.append(dataset_name)
            continue

        gene_names = read_adata.var.index.values
        all_gene_names.append(gene_names)
        success_loads.append(raw_dataset_name)
        
        all_h5ads.append(sp.csr_matrix(read_adata.X))
        all_cell_types.extend(read_adata.obs['cell_type'].values)
        #all_h5ads.append()
        #arr = np.memmap(path, shape=(ncells, 1280), dtype="float32", mode='r')
        #giant_matrix[current_row:current_row + ncells, :] = arr
        #current_row += ncells
    except:
        failed_loads.append(dataset_name)
        print('ERROR', dataset_name)


Progress:  21%|████████████████████████████▊                                                                                                           | 39/184 [06:01<07:53,  3.27s/file]

### Save all intermediate data files

In [ ]:
data_matrices = sp.vstack(all_h5ads)
sp.save_npz('fig4_raw_counts.npz', data_matrices)

In [23]:
all_datasets_list = []
for h5ad, dataset_ in zip(all_h5ads, np.unique(all_datasets)):
    all_datasets_list.extend([dataset_]*h5ad.shape[0])

In [ ]:
var_df = pd.DataFrame()
var_df.index = all_gene_names[0]
data_matrices_h5ad.var = var_df

In [ ]:
data_matrices_h5ad.obs['cell_type'] = all_cell_types
data_matrices_h5ad.obs['dataset'] = all_datasets_list

In [ ]:
np.save('fig4_cell_types.npy', all_cell_types)
np.save('fig4_datasets.npy', all_datasets_list)

In [25]:
data_matrices.shape

(19284565, 5704)

In [ ]:
data_matrices_h5ad.write_h5ad('fig4_raw_counts.h5ad')

### Other Metadata

In [64]:
all_datasets = []

for itr, x in enumerate(success_loads):
    all_datasets.extend([x]*all_lengths[itr])
    
np.save('all_datasets.npy', all_datasets)

all_lengths = [len(x) for x in all_h5ads]
all_lengths = np.hstack(all_lengths)
np.save('all_lengths.npy', all_cell_types)

In [49]:
all_cell_types = [x.obs['cell_type'].values for x in all_h5ads]
all_cell_types = np.hstack(all_cell_types)

np.save('all_cell_types.npy', all_cell_types)
np.save('success_loads.npy', success_loads)

In [36]:
shared_set = set(all_gene_names[0])
test = set(all_gene_names[0])

for itr, x in enumerate(all_gene_names):
    test = shared_set.intersection(set(x))
    if len(test)>100:
        shared_set = shared_set.intersection(set(x))
        test = set(shared_set)
    else:
        print(itr)
        
np.save('cxg_shared_genes.npy', list(shared_set))

3
6
16
17
20
37
58
90
94
103
113
114
121
132
160
162
167
211
218
270
275


### Add in tabula sapiens dataset - new donors

In [31]:
## Use updated gene list based on Tabula Sapiens
all_gene_names_edit = [item for item in all_gene_names if item != 'PHB']

In [ ]:
data_matrices_h5ad.X.max()

In [2]:
data_matrices_h5ad = sc.read_h5ad('fig4_raw_counts.h5ad')
data_matrices_h5ad = data_matrices_h5ad[:, all_gene_names_edit]

In [3]:
tabula_sapiens = sc.read_h5ad('/lfs/local/0/yanay/uce_temp/new_tabula_sapiens_ep_4_sn_125827_nlayers_4_sample_size_1024.h5ad')
tabula_sapiens = tabula_sapiens[:, all_gene_names_edit]

In [37]:
tabula_sapiens.obs['dataset']='TS'
tabula_sapiens = tabula_sapiens[tabula_sapiens.obs['donor'].isin([f'TSP{x}' for x in np.arange(16,31)])]
tabula_sapiens.obs['cell_type'] = tabula_sapiens.obs['cell_ontology_class'].values 

/tmp/user/20138/ipykernel_3413270/561214536.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  tabula_sapiens.obs['dataset']='TS'


In [44]:
adata_concat = sc.concat([data_matrices_h5ad, tabula_sapiens])

In [45]:
adata_concat

AnnData object with n_obs × n_vars = 19865995 × 5703
    obs: 'cell_type', 'dataset'

### Normalize data and compute centroids

In [49]:
sc.pp.normalize_total(adata_concat)
sc.pp.log1p(adata_concat)

### Now after normalization separate out CxG centroids from TS

In [51]:
cxg_adata = adata_concat[adata_concat.obs['dataset']!='TS']

In [52]:
cxg_adata

View of AnnData object with n_obs × n_vars = 19284565 × 5703
    obs: 'cell_type', 'dataset'
    uns: 'log1p'

In [57]:
all_cell_types = cxg_adata.obs['cell_type'].values
all_cell_types_uq = cxg_adata.obs['cell_type'].unique()

In [68]:
tabula_sapiens.obs['cell_type']

TSP30_Ovary_NA_SS3_Blue_B105390_LiveDead_D9       smooth muscle cell
TSP30_Ovary_NA_SS3_Blue_B105394_LiveDead_E22        endothelial cell
TSP30_Ovary_NA_SS3_Blue_B105394_LiveDead_P4       smooth muscle cell
TSP30_Ovary_NA_SS3_B004411_B105393_LiveDead_I1    smooth muscle cell
TSP30_Ovary_NA_SS3_Blue_B105394_LiveDead_O4       smooth muscle cell
                                                         ...        
TSP25_FAT_Brown_10X_1_6_TTTGTTGCAAAGCACG                      b cell
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTAAGTCAA                      b cell
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTATACCCA                  fibroblast
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTGTGTCCG                      b cell
TSP25_FAT_Brown_10X_1_6_TTTGTTGTCAAGCGTT          smooth muscle cell
Name: cell_type, Length: 581430, dtype: category
Categories (162, object): ['acinar cell', 'acinar cell of salivary gland', 'activated cd4-positive, alpha-beta t cell', 'activated cd8-positive, alpha-beta t cell', ..., 'vascular associated s

TSP30_Ovary_NA_SS3_Blue_B105390_LiveDead_D9       Ovary
TSP30_Ovary_NA_SS3_Blue_B105394_LiveDead_E22      Ovary
TSP30_Ovary_NA_SS3_Blue_B105394_LiveDead_P4       Ovary
TSP30_Ovary_NA_SS3_B004411_B105393_LiveDead_I1    Ovary
TSP30_Ovary_NA_SS3_Blue_B105394_LiveDead_O4       Ovary
                                                  ...  
TSP25_FAT_Brown_10X_1_6_TTTGTTGCAAAGCACG            Fat
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTAAGTCAA            Fat
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTATACCCA            Fat
TSP25_FAT_Brown_10X_1_6_TTTGTTGGTGTGTCCG            Fat
TSP25_FAT_Brown_10X_1_6_TTTGTTGTCAAGCGTT            Fat
Name: tissue, Length: 581430, dtype: category
Categories (27, object): ['Bladder', 'Blood', 'Bone_Marrow', 'Ear', ..., 'Tongue', 'Trachea', 'Uterus', 'Vasculature']

In [58]:
centroids = {}
for c in tqdm(all_cell_types_uq, total=len(all_cell_types_uq)):
    idxs = np.where(all_cell_types==c)
    subset_matrix = cxg_adata[idxs].X.toarray()
    centroids[c] = np.mean(subset_matrix,0)
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476/476 [05:44<00:00,  1.38it/s]


In [59]:
all_centroids = pd.DataFrame.from_dict(centroids)
all_centroids = all_centroids.T
all_centroids.columns = cxg_adata.var.index.values

In [60]:
all_centroids.to_csv('fig4_centroids_raw.csv')

### Now compute centroids for TS

In [61]:
tabula_adata = adata_concat[adata_concat.obs['dataset']=='TS']
tabula_adata.obs['tissue'] = tabula_sapiens.obs['tissue'].values

### Finally, compute alignment

In [62]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
results_dict = {}

In [63]:
tissue = 'Bladder'

In [93]:
def get_nn_results(concat_centroids, num_nbrs=[2], max_possible=None):
    
    results_dict = {}
    for n_neighbors in num_nbrs:

        # Create a NearestNeighbors model
        nn_model = NearestNeighbors(n_neighbors=n_neighbors)

        # Fit the model to your data
        nn_model.fit(concat_centroids)

        # Find the 1 nearest neighbor for each row
        distances, indices = nn_model.kneighbors(concat_centroids)

        # Create a new DataFrame to store the nearest neighbors and distances
        nearest_neighbors_dict = {concat_centroids.index[i]: np.array(concat_centroids.index)[indices[i, 1:n_neighbors]] 
                              for i in range(len(concat_centroids))
                              if '_TS' in concat_centroids.index[i]}

        matches = []
        not_matched = []

        for cell, neighbors in nearest_neighbors_dict.items():
            not_matched_flag = 1
            for n in neighbors:
                if cell.strip('_TS').lower() == n.lower():
                    not_matched_flag = 0
                    matches.append(cell)
                    break
            if not_matched_flag == 1:
                not_matched.append(cell)  

        not_matched = list(set(not_matched))
        if max_possible is not None:
            results_dict[n_neighbors] = len(matches)/max_possible
        else:
            results_dict[n_neighbors] = len(matches)
    
    return results_dict, not_matched

In [98]:
all_results = {}
num_nbrs = 4

for tissue in tabula_adata.obs['tissue'].unique():
    print(tissue)
    
    tabula_tissue = tabula_adata[tabula_adata.obs['tissue']==tissue]
    tabula_tissue_df = tabula_tissue.to_df()
    tabula_tissue_df['cell_type'] = tabula_tissue.obs['cell_type']
    tabula_tissue_centroids = tabula_tissue_df.groupby('cell_type').mean()
    tabula_tissue_centroids.index = [x+'_TS' for x in tabula_tissue_centroids.index.values]

    all_centroids.columns = all_centroids.columns.astype('str')
    tabula_tissue_centroids.columns = tabula_tissue_centroids.columns.astype('str')

    # Concatenate with IMA centroids
    concat_centroids = pd.concat([all_centroids, tabula_tissue_centroids])
    max_possible = len(set(tabula_tissue_df['cell_type'].unique().tolist()).intersection(
                        set(all_centroids.index.values)))
    
    results_dict, _ = get_nn_results(concat_centroids, 
                                     num_nbrs=[num_nbrs], 
                                     max_possible=max_possible)
    all_results[tissue] = results_dict

np.save(f'nn_{num_nbrs}_raw', all_results)

Ovary
Stomach
Fat
Thymus
Blood
Bladder
Small_Intestine
Salivary_Gland
Skin
Muscle
Bone_Marrow
Lung
Testis
Lymph_Node
Vasculature
Ear
Spleen
Large_Intestine
Trachea
Prostate
Uterus
Pancreas
Eye
Liver
Mammary
Heart
Tongue


In [97]:
all_results

{'Ovary': {2: 0.1},
 'Stomach': {2: 0.2727272727272727},
 'Fat': {2: 0.1111111111111111},
 'Thymus': {2: 0.11764705882352941},
 'Blood': {2: 0.07692307692307693},
 'Bladder': {2: 0.2},
 'Small_Intestine': {2: 0.16666666666666666},
 'Salivary_Gland': {2: 0.17647058823529413},
 'Skin': {2: 0.2222222222222222},
 'Muscle': {2: 0.047619047619047616},
 'Bone_Marrow': {2: 0.10526315789473684},
 'Lung': {2: 0.13043478260869565},
 'Testis': {2: 0.2},
 'Lymph_Node': {2: 0.07142857142857142},
 'Vasculature': {2: 0.09090909090909091},
 'Ear': {2: 0.16666666666666666},
 'Spleen': {2: 0.058823529411764705},
 'Large_Intestine': {2: 0.0},
 'Trachea': {2: 0.1875},
 'Prostate': {2: 0.3333333333333333},
 'Uterus': {2: 0.18181818181818182},
 'Pancreas': {2: 0.0},
 'Eye': {2: 0.17391304347826086},
 'Liver': {2: 0.1875},
 'Mammary': {2: 0.0},
 'Heart': {2: 0.0625},
 'Tongue': {2: 0.18181818181818182}}